In [ ]:
import pandas as pd
import transformers
import torch
import random
import pandas as pd
import random
import string
import numpy as np
import unidecode
from unidecode import unidecode
from wordcloud import STOPWORDS
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, f1_score
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoModel, AutoTokenizer, BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup, PreTrainedTokenizerFast
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import tensorflow as tf
from tqdm.notebook import tqdm
import numpy as np
import torch
transformers.logging.set_verbosity_debug()
from loguru import logger
logger.add("logging/finnlp_bert_large_undersampling_experiment_1.log", rotation="500 MB")

In [16]:
BATCH_SIZE = 15

In [17]:
logger.debug("<<<<<<<<<< INICIOU >>>>>>>>>>>")
brc = pd.read_csv("Banking_Regulation_Corpora_BRC_anonymous.csv" ,encoding="utf-8", low_memory=False, sep=";")
data_relevant = brc[brc['class'] == 1]
data_irrelevant = brc[brc['class'] == 0]

2024-03-01 20:10:23.414 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>


In [18]:
data_relevant.count()

class                   7846
department              7846
entry_date              7846
general_id              7846
normative_identifier    7846
publication_date        7846
regulatory_authority    7846
subject                 7846
subject_length          7846
subject_unique_words    7846
subject_words           7846
text                    7846
text_length             7846
text_unique_words       7846
text_words              7846
type                    7846
unique_document_id      7846
url                     7846
dtype: int64

In [19]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [20]:
data_relevant['class'] = data_relevant['class'].astype(str)

In [21]:
data_relevant = data_relevant.rename(columns={"class":"label"})

In [22]:
data_relevant.count()

label                   7714
department              7714
entry_date              7714
general_id              7714
normative_identifier    7714
publication_date        7714
regulatory_authority    7714
subject                 7714
subject_length          7714
subject_unique_words    7714
subject_words           7714
text                    7714
text_length             7714
text_unique_words       7714
text_words              7714
type                    7714
unique_document_id      7714
url                     7714
dtype: int64

In [23]:
data_irrelevant.count()

class                   54018
department              54018
entry_date              54018
general_id              54018
normative_identifier    54018
publication_date        54018
regulatory_authority    54018
subject                 54018
subject_length          54018
subject_unique_words    54018
subject_words           54018
text                    54018
text_length             54018
text_unique_words       54018
text_words              54018
type                    54018
unique_document_id      54018
url                     54018
dtype: int64

In [24]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [25]:
data_irrelevant['class'] = data_irrelevant['class'].astype(str)

In [26]:
data_irrelevant = data_irrelevant.rename(columns={"class":"label"})

In [27]:
data_irrelevant.count()

label                   53122
department              53122
entry_date              53122
general_id              53122
normative_identifier    53122
publication_date        53122
regulatory_authority    53122
subject                 53122
subject_length          53122
subject_unique_words    53122
subject_words           53122
text                    53122
text_length             53122
text_unique_words       53122
text_words              53122
type                    53122
unique_document_id      53122
url                     53122
dtype: int64

In [28]:
class Model_BERT():
    
    def model_path_def(self, name_model):
        if name_model == "BERTIMBAU_LARGE":
            model_path = MODEL_PATH
        elif name_model == "XXX":
            model_path = MODEL_PATH_NEW
        else:
            print("Nome de modelo inválido")
            model_path = "None"
            
        return model_path

    def tokenizer_model(self, model_path, df, token):
        tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

        # Tokenizing the training data
        encoded_data_train = tokenizer.batch_encode_plus(
            df[df.data_type=='train'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True, 
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )

        # Tokenizing test data
        encoded_data_val = tokenizer.batch_encode_plus(
            df[df.data_type=='val'].new_content.tolist(), 
            add_special_tokens=True, 
            return_attention_mask=True, 
            padding=True,
            max_length=token, 
            return_tensors='pt',
            truncation=True
        )


        input_ids_train = encoded_data_train['input_ids']
        attention_masks_train = encoded_data_train['attention_mask']
        labels_train = torch.tensor(df[df.data_type=='train'].label_num.values)

        input_ids_val = encoded_data_val['input_ids']
        attention_masks_val = encoded_data_val['attention_mask']
        labels_val = torch.tensor(df[df.data_type=='val'].label_num.values)

        dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
        dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
        
        return(dataset_train, dataset_val)

    def setup_model(self, model_path, dataset_train, dataset_val, epochs):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        batch_size = BATCH_SIZE
        dataloader_train = DataLoader(dataset_train, 
                                      sampler=RandomSampler(dataset_train), 
                                      batch_size=batch_size)
        dataloader_validation = DataLoader(dataset_val, 
                                           sampler=SequentialSampler(dataset_val), 
                                           batch_size=batch_size)
        optimizer = AdamW(model.parameters(),
                          lr=1e-5, 
                          eps=1e-8)

        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps=100,
                                                    num_training_steps=len(dataloader_train)*epochs)
        return(model, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device)


    def f1_score_func(self, preds, labels):
        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        
        scr = f1_score(labels_flat, preds_flat, average='binary')
        logger.info(f"f1_score_binary: {scr}")
        return scr

    def accuracy_per_class(self, preds, labels):
        label_dict_inverse = {v: k for k, v in LABEL_DICT.items()}
        

        preds_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()

        for label in np.unique(labels_flat):
            y_preds = preds_flat[labels_flat==label]
            y_true = labels_flat[labels_flat==label]
            print(f'Class: {label_dict_inverse[label]}')
            print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            print(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
            logger.info(f'Class: {label_dict_inverse[label]}')
            logger.info(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')
            logger.info(f'% Accuracy: {len(y_preds[y_preds==label])/len(y_true)}\n')
        
    def evaluate(self, model, dataloader_val, device):

        model.eval()

        loss_val_total = 0
        predictions, true_vals = [], []

        for batch in dataloader_val:

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            with torch.no_grad():        
                outputs = model(**inputs)

            loss = outputs[0]
            logits = outputs[1]
            loss_val_total += loss.item()

            logits = logits.detach().cpu().numpy()
            label_ids = inputs['labels'].cpu().numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        loss_val_avg = loss_val_total/len(dataloader_val) 

        predictions = np.concatenate(predictions, axis=0)
        true_vals = np.concatenate(true_vals, axis=0)

        logger.info(f"loss_val_avg, predictions, true_vals: {loss_val_avg}, {predictions}, {true_vals}")

        return loss_val_avg, predictions, true_vals

    def results(self, model_path, model_name, epochs, dataloader_validation, device, token):
        model = BertForSequenceClassification.from_pretrained(model_path,
                                                              num_labels=len(LABEL_DICT),
                                                              output_attentions=False,
                                                              output_hidden_states=False)

        model.to(device)

        model.load_state_dict(torch.load(f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epochs}.model', map_location=torch.device('cpu')))

        _, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
        Model_BERT().accuracy_per_class(predictions, true_vals)
        from sklearn import metrics
        preds_flat = np.argmax(predictions, axis=1).flatten()
        metrics_report = metrics.classification_report(true_vals,preds_flat, output_dict=True)
        logger.info(metrics_report)
                
        return metrics_report
    

    def training_model(self, model, model_name, dataloader_train, dataloader_validation, epochs, scheduler, optimizer, device, token):
        seed_val = 1
        random.seed(seed_val)
        np.random.seed(seed_val)
        torch.manual_seed(seed_val)
        torch.cuda.manual_seed_all(seed_val)
        epoch_array = []
        f1_array = []
        train_loss_array = []
        val_loss_array = []

        for epoch in tqdm(range(1, epochs+1)):

            model.train()

            loss_train_total = 0

            progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
            for batch in progress_bar:

                model.zero_grad()

                batch = tuple(b.to(device) for b in batch)

                inputs = {'input_ids':      batch[0],
                          'attention_mask': batch[1],
                          'labels':         batch[2],
                         }       

                outputs = model(**inputs)

                loss = outputs[0]
                loss_train_total += loss.item()
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

                optimizer.step()
                scheduler.step()

                progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


            torch.save(model.state_dict(),f'{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}.model')
            model.save_pretrained(f"{MODEL_SAVE_PATH}/token{token}_model_data_{model_name}_epoch_{epoch}")
            tqdm.write(f'\nEpoch {epoch}')
            print(f'\nEpoch {epoch}')
            logger.info(f'\nEpoch {epoch}')

            loss_train_avg = loss_train_total/len(dataloader_train)            
            tqdm.write(f'Training loss: {loss_train_avg}')
            logger.info(f'Training loss: {loss_train_avg}')

            val_loss, predictions, true_vals = Model_BERT().evaluate(model, dataloader_validation, device)
            val_f1 = Model_BERT().f1_score_func(predictions, true_vals)
            tqdm.write(f'Validation loss: {val_loss}')
            print(f'Validation loss: {val_loss}')
            logger.info(f'Validation loss: {val_loss}')
            
            tqdm.write(f'F1 Score (Weighted/Binary): {val_f1}')
            print(f'F1 Score (Weighted/Binary): {val_f1}')
            logger.info(f'F1 Score (Weighted/Binary): {val_f1}')
            
            epoch_array.append(epoch)
            f1_array.append(val_f1)
            val_loss_array.append(val_loss)
            train_loss_array.append(loss_train_avg)

        return(epoch_array, f1_array, val_loss_array, train_loss_array)
    
    def read_df(self, df):
        df['label_num'] = df.label.replace(LABEL_DICT)
        
        X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label_num.values, 
                                                  test_size=0.4, 
                                                  random_state=42, 
                                                  stratify=df.label.values)
        df['data_type'] = ['not_set']*df.shape[0]
        df.loc[X_train, 'data_type'] = 'train'
        df.loc[X_val, 'data_type'] = 'val'
        
        return(df)

    def call_setup(self, model_name, df, token, epochs):
        df = Model_BERT().read_df(df)
        model_path = Model_BERT().model_path_def(model_name)
        print(model_path)
        dataset_train, dataset_val = Model_BERT().tokenizer_model(model_path, df, token)
        model, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device = Model_BERT().setup_model(model_path, dataset_train, dataset_val, epochs)
        epoch_array, f1_array, val_loss_array, train_loss_array = Model_BERT().training_model(model, model_name, dataloader_train, dataloader_val, epochs, scheduler, optimizer, device, token)
        df_performa = pd.DataFrame()
        df_performa["epoch"] = epoch_array
        df_performa["f1"] = f1_array
        df_performa["val_loss"] = val_loss_array
        df_performa["train_loss"] = train_loss_array
        metrics_report = Model_BERT().results(model_path,model_name,epochs,dataloader_val, device, token)
        logger.info(f'Metric Report Call Setup: {metrics_report}')
        
        return metrics_report, df_performa

In [29]:
departments = ['DPT_25', 'DPT_3', 'DPT_2', 'DPT_9', 'DPT_5', 'DPT_16']

In [30]:
def cleaning(df,column):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    return df

In [ ]:
for department in departments:

    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == department].shape[0]
    logger.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {department}")
    logger.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == department].shape[0]} - {department}")
    
    data_relevant_board = data_relevant[data_relevant.department == department]
    logger.info(f"data_relevant_board: {data_relevant_board.shape} - {department}")
    
    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.
    data_irrelevant_board = data_irrelevant[data_irrelevant.department == department][:total_relevant_documents_of_the_board]
    logger.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {department}")

    df_ace_modelagem = pd.concat([data_irrelevant_board, data_relevant_board])
    
    logger.debug(f"Department: {department}")
    df_modelagem = df_ace_modelagem[df_ace_modelagem.department == department]

    df_modelagem = df_modelagem.dropna()
    df_modelagem = df_modelagem.reset_index(drop=True)
    df_modelagem = df_modelagem.rename(columns={"text":"content"})

    df_modelagem = cleaning(df_modelagem,"content")

    new_content = []
    for texto in df_modelagem.content:
        new_content.append(texto.replace("[BUSINESS NEED]", ""))
    df_modelagem["new_content"] = new_content
    
    LABEL_DICT = {'0': 0, '1': 1}

    MODEL_SAVE_PATH = "notebooks/testes/escoragem"
    BERTIMBAU_LARGE = "dataset/bert-large-portuguese-cased"
    BERTIMBAU_TOKENIZER = "tokenizer/BERTimbau_Tokenizer"

    MODEL_PATH = BERTIMBAU_LARGE
    MODEL_PATH_NEW = BERTIMBAU_LARGE

    tokenizer = BertTokenizer.from_pretrained(BERTIMBAU_TOKENIZER, do_lower_case=True)

    logger.debug(f"Department count samples: {df_modelagem.count()}")
    logger.debug(f"Department count samples class 1: {df_modelagem[df_modelagem.label == '1'].count()}")
    logger.debug(f"Department count samples class 0: {df_modelagem[df_modelagem.label == '0'].count()}")
    

    Model_BERT().call_setup("BERTIMBAU_LARGE", df_modelagem, 512, 5)

<h2>The output result is in the next cell. It was copied from the log file.</h2>

In [ ]:
# 2023-10-16 11:18:06.563 | DEBUG    | __main__:<module>:1 - <<<<<<<<<< INICIOU >>>>>>>>>>>
# 2023-10-16 11:18:16.831 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 819 - DPT_25
# 2023-10-16 11:18:16.838 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 4051 - DPT_25
# 2023-10-16 11:18:16.840 | INFO     | __main__:<module>:8 - data_relevant_board: (819, 18) - DPT_25
# 2023-10-16 11:18:16.845 | INFO     | __main__:<module>:12 - data_irrelevant_board: (819, 18) - DPT_25
# 2023-10-16 11:18:16.849 | DEBUG    | __main__:<module>:18 - Department: DPT_25
# 2023-10-16 11:18:20.221 | DEBUG    | __main__:<module>:53 - Department count samples: label                   1638
# department              1638
# entry_date              1638
# general_id              1638
# normative_identifier    1638
# publication_date        1638
# regulatory_authority    1638
# subject                 1638
# subject_length          1638
# subject_unique_words    1638
# subject_words           1638
# content                 1638
# text_length             1638
# text_unique_words       1638
# text_words              1638
# type                    1638
# unique_document_id      1638
# url                     1638
# new_content             1638
# dtype: int64
# 2023-10-16 11:18:20.225 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-16 11:18:20.229 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   819
# department              819
# entry_date              819
# general_id              819
# normative_identifier    819
# publication_date        819
# regulatory_authority    819
# subject                 819
# subject_length          819
# subject_unique_words    819
# subject_words           819
# content                 819
# text_length             819
# text_unique_words       819
# text_words              819
# type                    819
# unique_document_id      819
# url                     819
# new_content             819
# dtype: int64
# 2023-10-16 11:21:02.123 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 11:21:02.124 | INFO     | __main__:training_model:214 - Training loss: 0.5665207931941206
# 2023-10-16 11:21:21.453 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.44458932463418355, [[-0.56939614  1.0990589 ]

# 2023-10-16 11:21:21.456 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8
# 2023-10-16 11:21:21.458 | INFO     | __main__:training_model:220 - Validation loss: 0.44458932463418355
# 2023-10-16 11:21:21.459 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8
# 2023-10-16 11:23:00.190 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 11:23:00.193 | INFO     | __main__:training_model:214 - Training loss: 0.42923686405022937
# 2023-10-16 11:23:19.594 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.38967092453756114, [[-1.2045168   1.2743595 ]

# 2023-10-16 11:23:19.598 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8215488215488216
# 2023-10-16 11:23:19.600 | INFO     | __main__:training_model:220 - Validation loss: 0.38967092453756114
# 2023-10-16 11:23:19.601 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8215488215488216
# 2023-10-16 11:24:58.164 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 11:24:58.166 | INFO     | __main__:training_model:214 - Training loss: 0.33622951049244765
# 2023-10-16 11:25:17.584 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3164817977019332, [[-1.8283786   1.8242496 ]

# 2023-10-16 11:25:17.588 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8760064412238326
# 2023-10-16 11:25:17.589 | INFO     | __main__:training_model:220 - Validation loss: 0.3164817977019332
# 2023-10-16 11:25:17.591 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8760064412238326
# 2023-10-16 11:26:53.750 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 11:26:53.751 | INFO     | __main__:training_model:214 - Training loss: 0.2653636721837701
# 2023-10-16 11:27:13.212 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3078154611655257, [[-2.4466856   2.2146797 ]

# 2023-10-16 11:27:13.216 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8766233766233766
# 2023-10-16 11:27:13.217 | INFO     | __main__:training_model:220 - Validation loss: 0.3078154611655257
# 2023-10-16 11:27:13.218 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8766233766233766
# 2023-10-16 11:28:51.337 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 11:28:51.338 | INFO     | __main__:training_model:214 - Training loss: 0.20343350551345132
# 2023-10-16 11:29:10.781 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3046977175399661, [[-2.3351874  2.1522033]

# 2023-10-16 11:29:10.784 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8817891373801916
# 2023-10-16 11:29:10.785 | INFO     | __main__:training_model:220 - Validation loss: 0.3046977175399661
# 2023-10-16 11:29:10.786 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8817891373801916
# 2023-10-16 11:29:34.677 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3046977175399661, [[-2.3351874  2.1522033]

# 2023-10-16 11:29:34.679 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 11:29:34.680 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 306/328
# 2023-10-16 11:29:34.681 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9329268292682927

# 2023-10-16 11:29:34.682 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 11:29:34.683 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 276/328
# 2023-10-16 11:29:34.683 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8414634146341463

# 2023-10-16 11:29:34.691 | INFO     | __main__:results:157 - {'0': {'precision': 0.8547486033519553, 'recall': 0.9329268292682927, 'f1-score': 0.8921282798833818, 'support': 328}, '1': {'precision': 0.9261744966442953, 'recall': 0.8414634146341463, 'f1-score': 0.8817891373801916, 'support': 328}, 'accuracy': 0.8871951219512195, 'macro avg': {'precision': 0.8904615499981253, 'recall': 0.8871951219512195, 'f1-score': 0.8869587086317867, 'support': 656}, 'weighted avg': {'precision': 0.8904615499981253, 'recall': 0.8871951219512195, 'f1-score': 0.8869587086317866, 'support': 656}}
# 2023-10-16 11:29:34.694 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8547486033519553, 'recall': 0.9329268292682927, 'f1-score': 0.8921282798833818, 'support': 328}, '1': {'precision': 0.9261744966442953, 'recall': 0.8414634146341463, 'f1-score': 0.8817891373801916, 'support': 328}, 'accuracy': 0.8871951219512195, 'macro avg': {'precision': 0.8904615499981253, 'recall': 0.8871951219512195, 'f1-score': 0.8869587086317867, 'support': 656}, 'weighted avg': {'precision': 0.8904615499981253, 'recall': 0.8871951219512195, 'f1-score': 0.8869587086317866, 'support': 656}}
# 2023-10-16 11:29:34.701 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 688 - DPT_3
# 2023-10-16 11:29:34.708 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 1790 - DPT_3
# 2023-10-16 11:29:34.710 | INFO     | __main__:<module>:8 - data_relevant_board: (688, 18) - DPT_3
# 2023-10-16 11:29:34.715 | INFO     | __main__:<module>:12 - data_irrelevant_board: (688, 18) - DPT_3
# 2023-10-16 11:29:34.719 | DEBUG    | __main__:<module>:18 - Department: DPT_3
# 2023-10-16 11:29:47.921 | DEBUG    | __main__:<module>:53 - Department count samples: label                   1376
# department              1376
# entry_date              1376
# general_id              1376
# normative_identifier    1376
# publication_date        1376
# regulatory_authority    1376
# subject                 1376
# subject_length          1376
# subject_unique_words    1376
# subject_words           1376
# content                 1376
# text_length             1376
# text_unique_words       1376
# text_words              1376
# type                    1376
# unique_document_id      1376
# url                     1376
# new_content             1376
# dtype: int64
# 2023-10-16 11:29:47.926 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-16 11:29:47.929 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   688
# department              688
# entry_date              688
# general_id              688
# normative_identifier    688
# publication_date        688
# regulatory_authority    688
# subject                 688
# subject_length          688
# subject_unique_words    688
# subject_words           688
# content                 688
# text_length             688
# text_unique_words       688
# text_words              688
# type                    688
# unique_document_id      688
# url                     688
# new_content             688
# dtype: int64
# 2023-10-16 11:34:42.635 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 11:34:42.637 | INFO     | __main__:training_model:214 - Training loss: 0.6929952404715798
# 2023-10-16 11:34:58.900 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.6773132069690807, [[ 0.30348426  0.02432838]

# 2023-10-16 11:34:58.904 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.43478260869565216
# 2023-10-16 11:34:58.905 | INFO     | __main__:training_model:220 - Validation loss: 0.6773132069690807
# 2023-10-16 11:34:58.906 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.43478260869565216
# 2023-10-16 11:36:22.524 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 11:36:22.525 | INFO     | __main__:training_model:214 - Training loss: 0.6551436662673951
# 2023-10-16 11:36:38.822 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5856909244447142, [[ 0.08722901  0.04988722]

# 2023-10-16 11:36:38.826 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6975881261595547
# 2023-10-16 11:36:38.827 | INFO     | __main__:training_model:220 - Validation loss: 0.5856909244447142
# 2023-10-16 11:36:38.828 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6975881261595547
# 2023-10-16 11:38:02.817 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 11:38:02.819 | INFO     | __main__:training_model:214 - Training loss: 0.49475210254842583
# 2023-10-16 11:38:19.126 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4851993855995101, [[ 0.59804356 -0.36762857]

# 2023-10-16 11:38:19.130 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7717391304347825
# 2023-10-16 11:38:19.131 | INFO     | __main__:training_model:220 - Validation loss: 0.4851993855995101
# 2023-10-16 11:38:19.132 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7717391304347825
# 2023-10-16 11:39:43.016 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 11:39:43.017 | INFO     | __main__:training_model:214 - Training loss: 0.36618411595171146
# 2023-10-16 11:39:59.325 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4764693441825944, [[ 0.49186736 -0.36016956]

# 2023-10-16 11:39:59.329 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.783050847457627
# 2023-10-16 11:39:59.330 | INFO     | __main__:training_model:220 - Validation loss: 0.4764693441825944
# 2023-10-16 11:39:59.331 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.783050847457627
# 2023-10-16 11:41:23.829 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 11:41:23.830 | INFO     | __main__:training_model:214 - Training loss: 0.29054132794792004
# 2023-10-16 11:41:40.143 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.48134107897813255, [[ 1.4834504  -0.56097406]

# 2023-10-16 11:41:40.147 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.771266540642722
# 2023-10-16 11:41:40.148 | INFO     | __main__:training_model:220 - Validation loss: 0.48134107897813255
# 2023-10-16 11:41:40.150 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.771266540642722
# 2023-10-16 11:42:00.504 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.48134107897813255, [[ 1.4834504  -0.56097406]

# 2023-10-16 11:42:00.506 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 11:42:00.507 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 226/276
# 2023-10-16 11:42:00.508 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8188405797101449

# 2023-10-16 11:42:00.508 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 11:42:00.509 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 204/275
# 2023-10-16 11:42:00.510 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.7418181818181818

# 2023-10-16 11:42:00.518 | INFO     | __main__:results:157 - {'0': {'precision': 0.7609427609427609, 'recall': 0.8188405797101449, 'f1-score': 0.7888307155322862, 'support': 276}, '1': {'precision': 0.8031496062992126, 'recall': 0.7418181818181818, 'f1-score': 0.771266540642722, 'support': 275}, 'accuracy': 0.7803992740471869, 'macro avg': {'precision': 0.7820461836209868, 'recall': 0.7803293807641634, 'f1-score': 0.7800486280875041, 'support': 551}, 'weighted avg': {'precision': 0.7820078833983403, 'recall': 0.7803992740471869, 'f1-score': 0.7800645665402169, 'support': 551}}
# 2023-10-16 11:42:00.520 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.7609427609427609, 'recall': 0.8188405797101449, 'f1-score': 0.7888307155322862, 'support': 276}, '1': {'precision': 0.8031496062992126, 'recall': 0.7418181818181818, 'f1-score': 0.771266540642722, 'support': 275}, 'accuracy': 0.7803992740471869, 'macro avg': {'precision': 0.7820461836209868, 'recall': 0.7803293807641634, 'f1-score': 0.7800486280875041, 'support': 551}, 'weighted avg': {'precision': 0.7820078833983403, 'recall': 0.7803992740471869, 'f1-score': 0.7800645665402169, 'support': 551}}
# 2023-10-16 11:42:00.527 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 683 - DPT_2
# 2023-10-16 11:42:00.534 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 5234 - DPT_2
# 2023-10-16 11:42:00.536 | INFO     | __main__:<module>:8 - data_relevant_board: (683, 18) - DPT_2
# 2023-10-16 11:42:00.542 | INFO     | __main__:<module>:12 - data_irrelevant_board: (683, 18) - DPT_2
# 2023-10-16 11:42:00.545 | DEBUG    | __main__:<module>:18 - Department: DPT_2
# 2023-10-16 11:42:08.529 | DEBUG    | __main__:<module>:53 - Department count samples: label                   1366
# department              1366
# entry_date              1366
# general_id              1366
# normative_identifier    1366
# publication_date        1366
# regulatory_authority    1366
# subject                 1366
# subject_length          1366
# subject_unique_words    1366
# subject_words           1366
# content                 1366
# text_length             1366
# text_unique_words       1366
# text_words              1366
# type                    1366
# unique_document_id      1366
# url                     1366
# new_content             1366
# dtype: int64
# 2023-10-16 11:42:08.533 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-16 11:42:08.536 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   683
# department              683
# entry_date              683
# general_id              683
# normative_identifier    683
# publication_date        683
# regulatory_authority    683
# subject                 683
# subject_length          683
# subject_unique_words    683
# subject_words           683
# content                 683
# text_length             683
# text_unique_words       683
# text_words              683
# type                    683
# unique_document_id      683
# url                     683
# new_content             683
# dtype: int64
# 2023-10-16 11:45:37.836 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 11:45:37.837 | INFO     | __main__:training_model:214 - Training loss: 0.6495405495166778
# 2023-10-16 11:45:53.980 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.533238364232553, [[-0.7536986  -0.3357893 ]

# 2023-10-16 11:45:53.983 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6972860125260961
# 2023-10-16 11:45:53.985 | INFO     | __main__:training_model:220 - Validation loss: 0.533238364232553
# 2023-10-16 11:45:53.986 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6972860125260961
# 2023-10-16 11:47:17.688 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 11:47:17.690 | INFO     | __main__:training_model:214 - Training loss: 0.4142222485759042
# 2023-10-16 11:47:33.861 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4689302623674676, [[-1.9490296   0.34441915]

# 2023-10-16 11:47:33.865 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8083067092651757
# 2023-10-16 11:47:33.867 | INFO     | __main__:training_model:220 - Validation loss: 0.4689302623674676
# 2023-10-16 11:47:33.868 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8083067092651757
# 2023-10-16 11:48:54.737 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 11:48:54.738 | INFO     | __main__:training_model:214 - Training loss: 0.27152059091763064
# 2023-10-16 11:49:10.950 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.33486482098295883, [[-1.442017    0.17667153]

# 2023-10-16 11:49:10.954 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8602941176470588
# 2023-10-16 11:49:10.956 | INFO     | __main__:training_model:220 - Validation loss: 0.33486482098295883
# 2023-10-16 11:49:10.957 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8602941176470588
# 2023-10-16 11:50:34.193 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 11:50:34.194 | INFO     | __main__:training_model:214 - Training loss: 0.16704543331129984
# 2023-10-16 11:50:50.389 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3773621565888862, [[ 0.12286378 -0.5268767 ]

# 2023-10-16 11:50:50.393 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8671586715867159
# 2023-10-16 11:50:50.394 | INFO     | __main__:training_model:220 - Validation loss: 0.3773621565888862
# 2023-10-16 11:50:50.395 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8671586715867159
# 2023-10-16 11:52:11.354 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 11:52:11.356 | INFO     | __main__:training_model:214 - Training loss: 0.09682298820804466
# 2023-10-16 11:52:27.576 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3895244235172868, [[-1.0677322   0.00426753]

# 2023-10-16 11:52:27.580 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8664259927797835
# 2023-10-16 11:52:27.581 | INFO     | __main__:training_model:220 - Validation loss: 0.3895244235172868
# 2023-10-16 11:52:27.582 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8664259927797835
# 2023-10-16 11:52:47.920 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3895244235172868, [[-1.0677322   0.00426753]

# 2023-10-16 11:52:47.922 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 11:52:47.922 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 233/274
# 2023-10-16 11:52:47.923 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8503649635036497

# 2023-10-16 11:52:47.924 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 11:52:47.925 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 240/273
# 2023-10-16 11:52:47.926 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8791208791208791

# 2023-10-16 11:52:47.934 | INFO     | __main__:results:157 - {'0': {'precision': 0.8759398496240601, 'recall': 0.8503649635036497, 'f1-score': 0.8629629629629629, 'support': 274}, '1': {'precision': 0.8540925266903915, 'recall': 0.8791208791208791, 'f1-score': 0.8664259927797835, 'support': 273}, 'accuracy': 0.8647166361974405, 'macro avg': {'precision': 0.8650161881572258, 'recall': 0.8647429213122644, 'f1-score': 0.8646944778713732, 'support': 547}, 'weighted avg': {'precision': 0.8650361582878782, 'recall': 0.8647166361974405, 'f1-score': 0.8646913123962208, 'support': 547}}
# 2023-10-16 11:52:47.937 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8759398496240601, 'recall': 0.8503649635036497, 'f1-score': 0.8629629629629629, 'support': 274}, '1': {'precision': 0.8540925266903915, 'recall': 0.8791208791208791, 'f1-score': 0.8664259927797835, 'support': 273}, 'accuracy': 0.8647166361974405, 'macro avg': {'precision': 0.8650161881572258, 'recall': 0.8647429213122644, 'f1-score': 0.8646944778713732, 'support': 547}, 'weighted avg': {'precision': 0.8650361582878782, 'recall': 0.8647166361974405, 'f1-score': 0.8646913123962208, 'support': 547}}
# 2023-10-16 11:52:47.945 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 600 - DPT_9
# 2023-10-16 11:52:47.951 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 3619 - DPT_9
# 2023-10-16 11:52:47.953 | INFO     | __main__:<module>:8 - data_relevant_board: (600, 18) - DPT_9
# 2023-10-16 11:52:47.959 | INFO     | __main__:<module>:12 - data_irrelevant_board: (600, 18) - DPT_9
# 2023-10-16 11:52:47.963 | DEBUG    | __main__:<module>:18 - Department: DPT_9
# 2023-10-16 11:52:50.370 | DEBUG    | __main__:<module>:53 - Department count samples: label                   1200
# department              1200
# entry_date              1200
# general_id              1200
# normative_identifier    1200
# publication_date        1200
# regulatory_authority    1200
# subject                 1200
# subject_length          1200
# subject_unique_words    1200
# subject_words           1200
# content                 1200
# text_length             1200
# text_unique_words       1200
# text_words              1200
# type                    1200
# unique_document_id      1200
# url                     1200
# new_content             1200
# dtype: int64
# 2023-10-16 11:52:50.374 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-16 11:52:50.377 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   600
# department              600
# entry_date              600
# general_id              600
# normative_identifier    600
# publication_date        600
# regulatory_authority    600
# subject                 600
# subject_length          600
# subject_unique_words    600
# subject_words           600
# content                 600
# text_length             600
# text_unique_words       600
# text_words              600
# type                    600
# unique_document_id      600
# url                     600
# new_content             600
# dtype: int64
# 2023-10-16 11:54:45.611 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 11:54:45.612 | INFO     | __main__:training_model:214 - Training loss: 0.6898500298460325
# 2023-10-16 11:54:59.792 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5837405091151595, [[ 0.14918342  0.27164683]
 
# 2023-10-16 11:54:59.796 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7304347826086957
# 2023-10-16 11:54:59.797 | INFO     | __main__:training_model:220 - Validation loss: 0.5837405091151595
# 2023-10-16 11:54:59.798 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7304347826086957
# 2023-10-16 11:56:11.503 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 11:56:11.504 | INFO     | __main__:training_model:214 - Training loss: 0.4294491931796074
# 2023-10-16 11:56:25.719 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2984606445534155, [[-6.74726546e-01  5.34439087e-01]

# 2023-10-16 11:56:25.722 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8698884758364313
# 2023-10-16 11:56:25.724 | INFO     | __main__:training_model:220 - Validation loss: 0.2984606445534155
# 2023-10-16 11:56:25.725 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8698884758364313
# 2023-10-16 11:57:39.641 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 11:57:39.643 | INFO     | __main__:training_model:214 - Training loss: 0.27194326914226014
# 2023-10-16 11:57:53.840 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.24180243522278033, [[-2.9578659e-01  2.8419337e-01]
 
# 2023-10-16 11:57:53.843 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8857715430861723
# 2023-10-16 11:57:53.844 | INFO     | __main__:training_model:220 - Validation loss: 0.24180243522278033
# 2023-10-16 11:57:53.845 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8857715430861723
# 2023-10-16 11:59:05.325 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 11:59:05.326 | INFO     | __main__:training_model:214 - Training loss: 0.20259228123662373
# 2023-10-16 11:59:19.535 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.25940157774311956, [[-0.6473328   0.654946  ]

# 2023-10-16 11:59:19.539 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8951612903225807
# 2023-10-16 11:59:19.540 | INFO     | __main__:training_model:220 - Validation loss: 0.25940157774311956
# 2023-10-16 11:59:19.541 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8951612903225807
# 2023-10-16 12:00:33.555 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 12:00:33.557 | INFO     | __main__:training_model:214 - Training loss: 0.14043699767595777
# 2023-10-16 12:00:47.763 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.27973570268659387, [[-0.8096373   0.8088591 ]
 
# 2023-10-16 12:00:47.767 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8928571428571429
# 2023-10-16 12:00:47.768 | INFO     | __main__:training_model:220 - Validation loss: 0.27973570268659387
# 2023-10-16 12:00:47.769 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8928571428571429
# 2023-10-16 12:01:05.940 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.27973570268659387, [[-0.8096373   0.8088591 ]

# 2023-10-16 12:01:05.942 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 12:01:05.942 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 201/240
# 2023-10-16 12:01:05.943 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8375

# 2023-10-16 12:01:05.944 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 12:01:05.944 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 225/240
# 2023-10-16 12:01:05.945 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9375

# 2023-10-16 12:01:05.953 | INFO     | __main__:results:157 - {'0': {'precision': 0.9305555555555556, 'recall': 0.8375, 'f1-score': 0.881578947368421, 'support': 240}, '1': {'precision': 0.8522727272727273, 'recall': 0.9375, 'f1-score': 0.8928571428571429, 'support': 240}, 'accuracy': 0.8875, 'macro avg': {'precision': 0.8914141414141414, 'recall': 0.8875, 'f1-score': 0.887218045112782, 'support': 480}, 'weighted avg': {'precision': 0.8914141414141414, 'recall': 0.8875, 'f1-score': 0.887218045112782, 'support': 480}}
# 2023-10-16 12:01:05.955 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9305555555555556, 'recall': 0.8375, 'f1-score': 0.881578947368421, 'support': 240}, '1': {'precision': 0.8522727272727273, 'recall': 0.9375, 'f1-score': 0.8928571428571429, 'support': 240}, 'accuracy': 0.8875, 'macro avg': {'precision': 0.8914141414141414, 'recall': 0.8875, 'f1-score': 0.887218045112782, 'support': 480}, 'weighted avg': {'precision': 0.8914141414141414, 'recall': 0.8875, 'f1-score': 0.887218045112782, 'support': 480}}
# 2023-10-16 12:01:05.962 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 534 - DPT_5
# 2023-10-16 12:01:05.967 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 2594 - DPT_5
# 2023-10-16 12:01:05.969 | INFO     | __main__:<module>:8 - data_relevant_board: (534, 18) - DPT_5
# 2023-10-16 12:01:05.975 | INFO     | __main__:<module>:12 - data_irrelevant_board: (534, 18) - DPT_5
# 2023-10-16 12:01:05.978 | DEBUG    | __main__:<module>:18 - Department: DPT_5
# 2023-10-16 12:01:09.668 | DEBUG    | __main__:<module>:53 - Department count samples: label                   1068
# department              1068
# entry_date              1068
# general_id              1068
# normative_identifier    1068
# publication_date        1068
# regulatory_authority    1068
# subject                 1068
# subject_length          1068
# subject_unique_words    1068
# subject_words           1068
# content                 1068
# text_length             1068
# text_unique_words       1068
# text_words              1068
# type                    1068
# unique_document_id      1068
# url                     1068
# new_content             1068
# dtype: int64
# 2023-10-16 12:01:09.672 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-16 12:01:09.675 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   534
# department              534
# entry_date              534
# general_id              534
# normative_identifier    534
# publication_date        534
# regulatory_authority    534
# subject                 534
# subject_length          534
# subject_unique_words    534
# subject_words           534
# content                 534
# text_length             534
# text_unique_words       534
# text_words              534
# type                    534
# unique_document_id      534
# url                     534
# new_content             534
# dtype: int64
# 2023-10-16 12:03:16.098 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 12:03:16.100 | INFO     | __main__:training_model:214 - Training loss: 0.6934657166170519
# 2023-10-16 12:03:28.773 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5880766634283394, [[ 5.12984097e-01  2.11001709e-01]

# 2023-10-16 12:03:28.776 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7208791208791209
# 2023-10-16 12:03:28.777 | INFO     | __main__:training_model:220 - Validation loss: 0.5880766634283394
# 2023-10-16 12:03:28.779 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7208791208791209
# 2023-10-16 12:04:35.540 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 12:04:35.541 | INFO     | __main__:training_model:214 - Training loss: 0.5372880641804185
# 2023-10-16 12:04:48.214 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4916709389152198, [[-5.33580482e-01  1.74760044e-01]

# 2023-10-16 12:04:48.218 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.7975951903807615
# 2023-10-16 12:04:48.219 | INFO     | __main__:training_model:220 - Validation loss: 0.4916709389152198
# 2023-10-16 12:04:48.220 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.7975951903807615
# 2023-10-16 12:05:52.577 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 12:05:52.578 | INFO     | __main__:training_model:214 - Training loss: 0.35578080561271935
# 2023-10-16 12:06:05.273 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.3540103050141499, [[-0.45018533  0.15728012]

# 2023-10-16 12:06:05.277 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8137931034482758
# 2023-10-16 12:06:05.279 | INFO     | __main__:training_model:220 - Validation loss: 0.3540103050141499
# 2023-10-16 12:06:05.280 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8137931034482758
# 2023-10-16 12:07:12.149 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 12:07:12.150 | INFO     | __main__:training_model:214 - Training loss: 0.22603321690545525
# 2023-10-16 12:07:24.831 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.37821456748221455, [[-1.14101827e+00  2.98565328e-01]

# 2023-10-16 12:07:24.834 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8347826086956522
# 2023-10-16 12:07:24.835 | INFO     | __main__:training_model:220 - Validation loss: 0.37821456748221455
# 2023-10-16 12:07:24.836 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8347826086956522
# 2023-10-16 12:08:31.827 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 12:08:31.829 | INFO     | __main__:training_model:214 - Training loss: 0.1468432017190512
# 2023-10-16 12:08:44.491 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.43002483425340776, [[-1.6764097   0.6225817 ]

# 2023-10-16 12:08:44.495 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8275862068965518
# 2023-10-16 12:08:44.496 | INFO     | __main__:training_model:220 - Validation loss: 0.43002483425340776
# 2023-10-16 12:08:44.498 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8275862068965518
# 2023-10-16 12:09:01.316 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.43002483425340776, [[-1.6764097   0.6225817 ]

# 2023-10-16 12:09:01.318 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 12:09:01.319 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 156/214
# 2023-10-16 12:09:01.320 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.7289719626168224

# 2023-10-16 12:09:01.321 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 12:09:01.322 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 192/214
# 2023-10-16 12:09:01.323 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.897196261682243

# 2023-10-16 12:09:01.331 | INFO     | __main__:results:157 - {'0': {'precision': 0.8764044943820225, 'recall': 0.7289719626168224, 'f1-score': 0.7959183673469389, 'support': 214}, '1': {'precision': 0.768, 'recall': 0.897196261682243, 'f1-score': 0.8275862068965518, 'support': 214}, 'accuracy': 0.8130841121495327, 'macro avg': {'precision': 0.8222022471910113, 'recall': 0.8130841121495327, 'f1-score': 0.8117522871217453, 'support': 428}, 'weighted avg': {'precision': 0.8222022471910112, 'recall': 0.8130841121495327, 'f1-score': 0.8117522871217454, 'support': 428}}
# 2023-10-16 12:09:01.333 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.8764044943820225, 'recall': 0.7289719626168224, 'f1-score': 0.7959183673469389, 'support': 214}, '1': {'precision': 0.768, 'recall': 0.897196261682243, 'f1-score': 0.8275862068965518, 'support': 214}, 'accuracy': 0.8130841121495327, 'macro avg': {'precision': 0.8222022471910113, 'recall': 0.8130841121495327, 'f1-score': 0.8117522871217453, 'support': 428}, 'weighted avg': {'precision': 0.8222022471910112, 'recall': 0.8130841121495327, 'f1-score': 0.8117522871217454, 'support': 428}}
# 2023-10-16 12:09:01.341 | INFO     | __main__:<module>:4 - total_relevant_documents_of_the_board: 439 - DPT_16
# 2023-10-16 12:09:01.346 | INFO     | __main__:<module>:5 - Total irrelevant documents of the department: 1558 - DPT_16
# 2023-10-16 12:09:01.348 | INFO     | __main__:<module>:8 - data_relevant_board: (439, 18) - DPT_16
# 2023-10-16 12:09:01.354 | INFO     | __main__:<module>:12 - data_irrelevant_board: (439, 18) - DPT_16
# 2023-10-16 12:09:01.358 | DEBUG    | __main__:<module>:18 - Department: DPT_16
# 2023-10-16 12:09:06.785 | DEBUG    | __main__:<module>:53 - Department count samples: label                   878
# department              878
# entry_date              878
# general_id              878
# normative_identifier    878
# publication_date        878
# regulatory_authority    878
# subject                 878
# subject_length          878
# subject_unique_words    878
# subject_words           878
# content                 878
# text_length             878
# text_unique_words       878
# text_words              878
# type                    878
# unique_document_id      878
# url                     878
# new_content             878
# dtype: int64
# 2023-10-16 12:09:06.789 | DEBUG    | __main__:<module>:54 - Department count samples class 1: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-16 12:09:06.793 | DEBUG    | __main__:<module>:55 - Department count samples class 0: label                   439
# department              439
# entry_date              439
# general_id              439
# normative_identifier    439
# publication_date        439
# regulatory_authority    439
# subject                 439
# subject_length          439
# subject_unique_words    439
# subject_words           439
# content                 439
# text_length             439
# text_unique_words       439
# text_words              439
# type                    439
# unique_document_id      439
# url                     439
# new_content             439
# dtype: int64
# 2023-10-16 12:11:30.450 | INFO     | __main__:training_model:210 - 
# Epoch 1
# 2023-10-16 12:11:30.453 | INFO     | __main__:training_model:214 - Training loss: 0.6689137096206347
# 2023-10-16 12:11:40.846 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.5383422747254372, [[-0.61664087 -0.26155   ]

# 2023-10-16 12:11:40.850 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8020050125313283
# 2023-10-16 12:11:40.851 | INFO     | __main__:training_model:220 - Validation loss: 0.5383422747254372
# 2023-10-16 12:11:40.852 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8020050125313283
# 2023-10-16 12:12:37.051 | INFO     | __main__:training_model:210 - 
# Epoch 2
# 2023-10-16 12:12:37.052 | INFO     | __main__:training_model:214 - Training loss: 0.5244079108039538
# 2023-10-16 12:12:47.453 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.4673820994794369, [[-0.771734   -0.33849487]

# 2023-10-16 12:12:47.456 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.6199261992619927
# 2023-10-16 12:12:47.458 | INFO     | __main__:training_model:220 - Validation loss: 0.4673820994794369
# 2023-10-16 12:12:47.459 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.6199261992619927
# 2023-10-16 12:13:41.471 | INFO     | __main__:training_model:210 - 
# Epoch 3
# 2023-10-16 12:13:41.472 | INFO     | __main__:training_model:214 - Training loss: 0.418528500944376
# 2023-10-16 12:13:51.900 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.32494894104699296, [[-0.13720173 -0.28249618]

# 2023-10-16 12:13:51.905 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8522727272727273
# 2023-10-16 12:13:51.906 | INFO     | __main__:training_model:220 - Validation loss: 0.32494894104699296
# 2023-10-16 12:13:51.907 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8522727272727273
# 2023-10-16 12:14:48.008 | INFO     | __main__:training_model:210 - 
# Epoch 4
# 2023-10-16 12:14:48.009 | INFO     | __main__:training_model:214 - Training loss: 0.32487695167462033
# 2023-10-16 12:14:58.419 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.32893385652763146, [[ 0.3284121  -0.29890972]

# 2023-10-16 12:14:58.423 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8650137741046832
# 2023-10-16 12:14:58.424 | INFO     | __main__:training_model:220 - Validation loss: 0.32893385652763146
# 2023-10-16 12:14:58.425 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8650137741046832
# 2023-10-16 12:15:55.090 | INFO     | __main__:training_model:210 - 
# Epoch 5
# 2023-10-16 12:15:55.092 | INFO     | __main__:training_model:214 - Training loss: 0.2721165236499574
# 2023-10-16 12:16:05.496 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2886545436146359, [[-0.840442   -0.1579147 ]

# 2023-10-16 12:16:05.500 | INFO     | __main__:f1_score_func:85 - f1_score_binary: 0.8804347826086957
# 2023-10-16 12:16:05.501 | INFO     | __main__:training_model:220 - Validation loss: 0.2886545436146359
# 2023-10-16 12:16:05.502 | INFO     | __main__:training_model:224 - F1 Score (Weighted/Binary): 0.8804347826086957
# 2023-10-16 12:16:19.847 | INFO     | __main__:evaluate:138 - loss_val_avg, predictions, true_vals: 0.2886545436146359, [[-0.840442   -0.1579147 ]

# 2023-10-16 12:16:19.849 | INFO     | __main__:accuracy_per_class:101 - Class: 0
# 2023-10-16 12:16:19.850 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 146/176
# 2023-10-16 12:16:19.851 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.8295454545454546

# 2023-10-16 12:16:19.852 | INFO     | __main__:accuracy_per_class:101 - Class: 1
# 2023-10-16 12:16:19.853 | INFO     | __main__:accuracy_per_class:102 - Accuracy: 162/176
# 2023-10-16 12:16:19.853 | INFO     | __main__:accuracy_per_class:103 - % Accuracy: 0.9204545454545454

# 2023-10-16 12:16:19.861 | INFO     | __main__:results:157 - {'0': {'precision': 0.9125, 'recall': 0.8295454545454546, 'f1-score': 0.869047619047619, 'support': 176}, '1': {'precision': 0.84375, 'recall': 0.9204545454545454, 'f1-score': 0.8804347826086957, 'support': 176}, 'accuracy': 0.875, 'macro avg': {'precision': 0.878125, 'recall': 0.875, 'f1-score': 0.8747412008281573, 'support': 352}, 'weighted avg': {'precision': 0.878125, 'recall': 0.875, 'f1-score': 0.8747412008281572, 'support': 352}}
# 2023-10-16 12:16:19.863 | INFO     | __main__:call_setup:260 - Metric Report Call Setup: {'0': {'precision': 0.9125, 'recall': 0.8295454545454546, 'f1-score': 0.869047619047619, 'support': 176}, '1': {'precision': 0.84375, 'recall': 0.9204545454545454, 'f1-score': 0.8804347826086957, 'support': 176}, 'accuracy': 0.875, 'macro avg': {'precision': 0.878125, 'recall': 0.875, 'f1-score': 0.8747412008281573, 'support': 352}, 'weighted avg': {'precision': 0.878125, 'recall': 0.875, 'f1-score': 0.8747412008281572, 'support': 352}}
